# RCT Data Preparation Pipeline

This notebook prepares the clinical trial data for the LightGBM prediction pipeline.

## Input Requirements:
- Excel file with clinical trial data
- Sheet: "250529_NSCLC" (skip first 2 rows)

## Output:
- Processed CSV file with required columns for the pipeline

## Required Columns for Pipeline:
- `rct_id`: RCT identifier (renamed from NCT_ID)
- `Arm`: "Control" or "Intervention" 
- `is_target_trial`: Boolean flag (1 for target RCT, 0 for others)
- `PFS_median_months`: Target variable
- Feature columns as specified in config

In [1]:
import pandas as pd
import numpy as np
import os

print("Data preparation pipeline initialized")

Data preparation pipeline initialized


## Step 1: Load Raw Data

In [2]:
# Load the Excel file
excel_file = "20250521_Trials for dev.xlsx"
sheet_name = "250529_NSCLC"
skip_rows = 2

print(f"Loading data from: {excel_file}")
print(f"Sheet: {sheet_name}")
print(f"Skipping first {skip_rows} rows")

df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=skip_rows)

print(f"\nLoaded data shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
df.head(2)

Loading data from: 20250521_Trials for dev.xlsx
Sheet: 250529_NSCLC
Skipping first 2 rows

Loaded data shape: (96, 52)
Columns: 52


,NCT_ID,Arm,to keep,Iteration,Pubmed Link,Downloaded Paper Link,Total Population,arm_n,intervention,existing?,...,need_to_be_dropped,NOTES,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,100
0,NCT02142738,Intervention,1.0,2025-05-29,https://pmc.ncbi.nlm.nih.gov/articles/PMC82800...,NaN,305.0,154,Pembrolizumab,0.0,...,0.0,(PD-L1) tumor proportion score of at least 50%...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Control,1.0,2025-05-29,NaN,NaN,NaN,151,platinum + pemetrexed or gemcitabine or paclit...,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 2: Basic Data Inspection

In [3]:
# Check key columns
print("Key columns inspection:")
print(f"NCT_ID column exists: {'NCT_ID' in df.columns}")
print(f"Arm column exists: {'Arm' in df.columns}")
print(f"PFS_median_months column exists: {'PFS_median_months' in df.columns}")

if 'Arm' in df.columns:
    print(f"\nArm column values:")
    print(df['Arm'].value_counts())

if 'NCT_ID' in df.columns:
    print(f"\nUnique RCT IDs: {df['NCT_ID'].nunique()}")
    print(f"Sample RCT IDs: {df['NCT_ID'].dropna().unique()[:5]}")

Key columns inspection:
NCT_ID column exists: True
Arm column exists: True
PFS_median_months column exists: True

Arm column values:
Arm
Intervention    46
Control         41
Name: count, dtype: int64

Unique RCT IDs: 41
Sample RCT IDs: ['NCT02142738' 'NCT02041533' 'NCT02657434' 'NCT03950674' 'NCT02367781']


## Step 3: Data Preprocessing

In [4]:
# Step 3.1: Rename NCT_ID to rct_id (required by pipeline)
if 'NCT_ID' in df.columns:
    df.rename(columns={'NCT_ID': 'rct_id'}, inplace=True)
    print("Renamed NCT_ID column to rct_id")
else:
    print("NCT_ID column not found")

# Step 3.2: Create target trial flag
target_rct_id = 'NCT02578680'  # Default target RCT
df['is_target_trial'] = df['rct_id'].apply(lambda x: 1 if x == target_rct_id else 0)

print(f"\nCreated is_target_trial flag for target RCT: {target_rct_id}")
print(f"Target trial count: {df['is_target_trial'].sum()}")
print(f"Similar trials count: {(df['is_target_trial'] == 0).sum()}")

Renamed NCT_ID column to rct_id

Created is_target_trial flag for target RCT: NCT02578680
Target trial count: 1
Similar trials count: 95


## Step 4: Data Quality Checks

In [5]:
# Check required columns for pipeline
required_columns = [
    'rct_id', 'Arm', 'is_target_trial', 'PFS_median_months'
]

print("Required columns check:")
for col in required_columns:
    exists = col in df.columns
    print(f"  {col}: {'OK' if exists else 'Not OK'}")
    
    if exists and col != 'is_target_trial':
        null_count = df[col].isnull().sum()
        print(f"    - Null values: {null_count} ({null_count/len(df)*100:.1f}%)")

# Check target variable
if 'PFS_median_months' in df.columns:
    pfs_stats = df['PFS_median_months'].describe()
    print(f"\nPFS_median_months statistics:")
    print(f"  Count: {pfs_stats['count']:.0f}")
    print(f"  Mean: {pfs_stats['mean']:.2f}")
    print(f"  Std: {pfs_stats['std']:.2f}")
    print(f"  Range: {pfs_stats['min']:.2f} - {pfs_stats['max']:.2f}")

Required columns check:
  rct_id: OK
    - Null values: 52 (54.2%)
  Arm: OK
    - Null values: 9 (9.4%)
  is_target_trial: OK
  PFS_median_months: OK
    - Null values: 31 (32.3%)

PFS_median_months statistics:
  Count: 65
  Mean: 6.74
  Std: 1.89
  Range: 4.20 - 12.10


## Step 5: Feature Columns Check

In [6]:
# Features as specified in the config
pipeline_features = [
    "age_median",
    "gender_male_percent", 
    "no_smoker_percent",
    "ecog_1",
    "brain_metastase_yes",
    "disease_stage_IV",
    "EGFR_positive_mutation",
    "Chemotherapy",
    "Targeted_Therapy", 
    "Immunotherapy",
    "Anti-angiogenic_Other"
]

print("Pipeline features availability:")
available_features = []
missing_features = []

for feature in pipeline_features:
    if feature in df.columns:
        available_features.append(feature)
        null_pct = df[feature].isnull().sum() / len(df) * 100
        print(f"{feature} (null: {null_pct:.1f}%)")
    else:
        missing_features.append(feature)
        print(f"{feature}")

print(f"\nSummary:")
print(f"  Available features: {len(available_features)}/{len(pipeline_features)}")
print(f"  Missing features: {missing_features}")

Pipeline features availability:
age_median (null: 62.5%)
gender_male_percent (null: 60.4%)
no_smoker_percent (null: 60.4%)
ecog_1 (null: 60.4%)
brain_metastase_yes (null: 66.7%)
disease_stage_IV (null: 78.1%)
EGFR_positive_mutation (null: 87.5%)
Chemotherapy (null: 38.5%)
Targeted_Therapy (null: 38.5%)
Immunotherapy (null: 38.5%)
Anti-angiogenic_Other (null: 40.6%)

Summary:
  Available features: 11/11
  Missing features: []


## Step 6: Arm Distribution Analysis

In [7]:
# Analyze arm distribution
if 'Arm' in df.columns:
    print("Arm distribution:")
    arm_counts = df['Arm'].value_counts()
    print(arm_counts)
    
    print(f"\nTarget RCT arm distribution:")
    target_arms = df[df['is_target_trial'] == 1]['Arm'].value_counts()
    print(target_arms)
    
    print(f"\nSimilar RCTs arm distribution:")
    similar_arms = df[df['is_target_trial'] == 0]['Arm'].value_counts()
    print(similar_arms)
    
    # Check which RCTs have both arms
    print(f"\nRCTs with both Control and Intervention arms:")
    rct_arm_counts = df.groupby('rct_id')['Arm'].nunique()
    both_arms_rcts = rct_arm_counts[rct_arm_counts == 2].index.tolist()
    print(f"  Count: {len(both_arms_rcts)}")
    if len(both_arms_rcts) > 0:
        print(f"  RCTs: {both_arms_rcts[:5]}...")  # Show first 5
else:
    print("❌ Arm column not found")

Arm distribution:
Arm
Intervention    46
Control         41
Name: count, dtype: int64

Target RCT arm distribution:
Arm
Intervention    1
Name: count, dtype: int64

Similar RCTs arm distribution:
Arm
Intervention    45
Control         41
Name: count, dtype: int64

RCTs with both Control and Intervention arms:
  Count: 1
  RCTs: ['Results NA']...


## Step 7: Save Processed Data

In [9]:
# Create data directory if it doesn't exist
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

# Save processed data
output_file = os.path.join(data_dir, "trial_data.csv")
df.to_csv(output_file, index=False)

print(f"Data saved to: {output_file}")
print(f"   Final shape: {df.shape}")
print(f"   Columns: {len(df.columns)}")

# Verify the saved file
test_df = pd.read_csv(output_file)
print(f"\nVerification: Loaded saved file shape: {test_df.shape}")

# Show sample of final data
print(f"\nSample of processed data:")
sample_cols = ['rct_id', 'Arm', 'is_target_trial', 'PFS_median_months'] + available_features[:3]
display(df[sample_cols].head())

Data saved to: data/trial_data.csv
   Final shape: (96, 53)
   Columns: 53

Verification: Loaded saved file shape: (96, 53)

Sample of processed data:


,rct_id,Arm,is_target_trial,PFS_median_months,age_median,gender_male_percent,no_smoker_percent
0,NCT02142738,Intervention,0,7.7,64.5,59.7,3.2
1,NaN,Control,0,5.5,66.0,62.9,12.6
2,NCT02041533,Intervention,0,4.2,63.0,68.0,11.0
3,NaN,Control,0,5.9,65.0,55.0,11.0
4,NaN,Intervention,0,9.7,NaN,NaN,NaN


## Step 8: Generate Pipeline Configuration

In [10]:
# Generate config for the pipeline
config = {
    "ftr_list": available_features,
    "target_variable": "PFS_median_months",
    "rct_id_col_name": "rct_id",
    "arm_type_col_name": "Arm",
    "is_target_rct": "is_target_trial",
    "data_path": output_file
}

# Save config
import json
config_file = "config_generated.json"
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ Configuration saved to: {config_file}")
print(f"\nGenerated config:")
print(json.dumps(config, indent=2))

✅ Configuration saved to: config_generated.json

Generated config:
{
  "ftr_list": [
    "age_median",
    "gender_male_percent",
    "no_smoker_percent",
    "ecog_1",
    "brain_metastase_yes",
    "disease_stage_IV",
    "EGFR_positive_mutation",
    "Chemotherapy",
    "Targeted_Therapy",
    "Immunotherapy",
    "Anti-angiogenic_Other"
  ],
  "target_variable": "PFS_median_months",
  "rct_id_col_name": "rct_id",
  "arm_type_col_name": "Arm",
  "is_target_rct": "is_target_trial",
  "data_path": "data/trial_data.csv"
}
